# Regression with Amazon SageMaker XGBoost algorithm
_**Distributed training for regression with Amazon SageMaker XGBoost script mode**_

---

## Contents
1. [Introduction](#Introduction)
2. [Setup](#Setup)
  1. [Fetching the dataset](#Fetching-the-dataset)
  2. [Data Ingestion](#Data-ingestion)
3. [Training the XGBoost model](#Training-the-XGBoost-model)
3. [Deploying the XGBoost model](#Deploying-the-XGBoost-model)

---

## Introduction

This notebook demonstrates the use of Amazon SageMaker XGBoost to train and host a regression model. [XGBoost (eXtreme Gradient Boosting)](https://xgboost.readthedocs.io) is a popular and efficient machine learning algorithm used for regression and classification tasks on tabular datasets. It implements a technique know as gradient boosting on trees, and performs remarkably well in machine learning competitions, and gets a lot of attention from customers. 

We use the [Abalone data](https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/regression.html), originally from the [UCI data repository](https://archive.ics.uci.edu/ml/datasets/abalone). More details about the original dataset can be found [here](https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.names).  In this libsvm converted version, the nominal feature (Male/Female/Infant) has been converted into a real valued feature as required by XGBoost. Age of abalone is to be predicted from eight physical measurements.  

---
## Setup

This notebook was tested in Amazon SageMaker Studio on a ml.t3.medium instance with Python 3 (Data Science) kernel.

Let's start by specifying:
1. The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
1. The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [40]:
%%time

import os
import boto3
import re
import sagemaker

# Get a SageMaker-compatible role used by this Notebook Instance.
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

### update below values appropriately ###
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/xgboost-pipe-dist-script'
#### 

print(region)

us-east-2
CPU times: user 82.6 ms, sys: 0 ns, total: 82.6 ms
Wall time: 156 ms


### Fetching the dataset

Following methods split the data into train/test/validation datasets and upload files to S3.

### Data ingestion

Next, we read the dataset from the existing repository into memory, for preprocessing prior to training. This processing could be done *in situ* by Amazon Athena, Apache Spark in Amazon EMR, Amazon Redshift, etc., assuming the dataset is present in the appropriate location. Then, the next step would be to transfer the data to S3 for use in training. For small datasets, such as this one, reading into memory isn't onerous, though it would be for larger datasets.

In [41]:
%%time

import pyarrow
import numpy as np
import pandas as pd
from sklearn.datasets import load_svmlight_file

s3 = boto3.client("s3")
# Download the dataset and load into a pandas dataframe
FILE_NAME = 'abalone.csv'
s3.download_file("sagemaker-sample-files", f"datasets/tabular/uci_abalone/abalone.csv", FILE_NAME)
feature_names=['Sex', 
               'Length', 
               'Diameter', 
               'Height', 
               'Whole weight', 
               'Shucked weight', 
               'Viscera weight', 
               'Shell weight', 
               'Rings']
data = pd.read_csv(FILE_NAME, 
                   header=None, 
                   names=feature_names)

# SageMaker XGBoost has the convention of label in the first column
data = data[feature_names[-1:] + feature_names[:-1]]
data["Sex"] = data["Sex"].astype("category").cat.codes

# Split the downloaded data into train/test dataframes
train, validation = np.split(data.sample(frac=1), [int(.8*len(data))])
train_0, train_1 = np.split(train.sample(frac=1), [int(.5*len(train))])

# requires PyArrow installed
train_0.to_parquet('abalone_train_0.parquet')
train_1.to_parquet('abalone_train_1.parquet')
validation.to_parquet('abalone_validation.parquet')

CPU times: user 56.7 ms, sys: 784 µs, total: 57.5 ms
Wall time: 284 ms


In [42]:
%%time

sagemaker.Session().upload_data('abalone_train_0.parquet', 
                                bucket=bucket, 
                                key_prefix=prefix+'/'+'train')

sagemaker.Session().upload_data('abalone_train_1.parquet', 
                                bucket=bucket, 
                                key_prefix=prefix+'/'+'train')

sagemaker.Session().upload_data('abalone_validation.parquet', 
                                bucket=bucket, 
                                key_prefix=prefix+'/'+'validation')

CPU times: user 86.6 ms, sys: 151 µs, total: 86.7 ms
Wall time: 301 ms


's3://sagemaker-us-east-2-240487350066/sagemaker/xgboost-pipe-dist-script/validation/abalone_validation.parquet'

## Create a XGBoost script to train with 

SageMaker can now run an XGboost script using the XGBoost estimator. When executed on SageMaker a number of helpful environment variables are available to access properties of the training environment, such as:

- `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to. Any artifacts saved in this folder are uploaded to S3 for model hosting after the training job completes.
- `SM_OUTPUT_DIR`: A string representing the filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.

Supposing two input channels, 'train' and 'validation', were used in the call to the XGBoost estimator's fit() method, the following environment variables will be set, following the format `SM_CHANNEL_[channel_name]`:

`SM_CHANNEL_TRAIN`: A string representing the path to the directory containing data in the 'train' channel
`SM_CHANNEL_VALIDATION`: Same as above, but for the 'validation' channel.

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to model_dir so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an argparse.ArgumentParser instance. For example, the script that we will run in this notebook is provided as the accompanying file (`abalone_pipe.py`).

Because the container imports your training script, always put your training code in a main guard `(if __name__=='__main__':)` so that the container does not inadvertently run your training code at the wrong point in execution.

For more information about training environment variables, please visit https://github.com/aws/sagemaker-containers.

## Training the XGBoost model

After setting training parameters, we kick off training, and poll for status until training is completed, which in this example, takes between few minutes.

To run our training script on SageMaker, we construct a sagemaker.xgboost.estimator.XGBoost estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script SageMaker runs for training and prediction.
* __role__: Role ARN
* __train_instance_type__ *(optional)*: The type of SageMaker instances for training. __Note__: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.
* __hyperparameters__ *(optional)*: A dictionary passed to the train function as hyperparameters.

In [43]:
hyperparams = {
        "max_depth": "5",
        "eta": "0.2",
        "gamma": "4",
        "min_child_weight": "6",
        "subsample": "0.7",
        "objective": "reg:squarederror",
        "num_round": "50",
        "verbosity": "2",
        "is_pipe_mode": True
}

instance_type = "ml.m5.2xlarge"
output_path = 's3://{}/{}/output'.format(bucket, prefix)
content_type = "parquet"
input_mode = "Pipe"

In [44]:
# Open Source distributed script mode
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

boto_session = boto3.Session(region_name=region)
session = Session(boto_session=boto_session)
script_path = 'abalone_pipe.py'

xgb_script_mode_estimator = XGBoost(
    entry_point=script_path,
    framework_version='1.2-1', # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=role,
    instance_count=2,
    instance_type=instance_type,
    output_path=output_path,
    input_mode=input_mode)

train_input = TrainingInput("s3://{}/{}/{}/".format(bucket, prefix, "train"), distribution='ShardedByS3Key', content_type=content_type)
validation_input = TrainingInput("s3://{}/{}/{}/".format(bucket, prefix, "validation"), distribution='FullyReplicated', content_type=content_type)

### Train XGBoost Estimator on abalone data 


Training is as simple as calling `fit` on the Estimator. This will start a SageMaker Training job that will download the data, invoke the entry point code (in the provided script file), and save any model artifacts that the script creates.

In [45]:
xgb_script_mode_estimator.fit({'train': train_input, 'validation': validation_input})

2021-04-23 17:33:12 Starting - Starting the training job...
2021-04-23 17:33:34 Starting - Launching requested ML instancesProfilerReport-1619199191: InProgress
......
2021-04-23 17:34:35 Starting - Preparing the instances for training......
2021-04-23 17:35:41 Downloading - Downloading input data
2021-04-23 17:35:41 Training - Downloading the training image...
2021-04-23 17:35:55 Training - Training image download completed. Training in progress.[2021-04-23 17:35:56.309 ip-10-0-198-51.us-east-2.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module abalone_pipe does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:s

## Deploying the XGBoost model

After training, we can use the estimator to create an Amazon SageMaker endpoint – a hosted and managed prediction service that we can use to perform inference.

You can also optionally specify other functions to customize the behavior of deserialization of the input request (`input_fn()`), serialization of the predictions (`output_fn()`), and how predictions are made (`predict_fn()`). The defaults work for our current use-case so we don’t need to define them.

In [46]:
predictor = xgb_script_mode_estimator.deploy(initial_instance_count=1, 
                                             instance_type="ml.m5.2xlarge")

---------------!

### (Optional) Delete the Endpoint

If you're done with this exercise, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint and avoid any charges from a stray instance being left on.

In [47]:
predictor.delete_endpoint()